# This notebook is for testing and Integration of fields_building_numerical_all.py with the develop branch

In [ ]:
import os
import time
import functools
from shutil import copy
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.units as u
import hcipy
import numpy as np
import pastis.util as util    
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC 

In [ ]:
root_dir = CONFIG_PASTIS.get('local', 'local_data_path')
output_dir = CONFIG_PASTIS.get('local', 'output_data_folder') #needs to be added to config_ini
coronagraph_design = 'small' #c user provides
overall_dir = output_dir + 'luvoirA-'+ coronagraph_design 
resDir = os.path.join(overall_dir, 'matrix_numerical')

In [ ]:
max_LO = CONFIG_PASTIS.getint('calibration', 'max_LO') 
max_MID = CONFIG_PASTIS.getint('calibration', 'max_MID')
max_HI = CONFIG_PASTIS.getint('calibration', 'max_HI')
num_DM_act = CONFIG_PASTIS.getint('calibration', 'num_DM_act') 

In [ ]:
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
wvln = CONFIG_PASTIS.getfloat('LUVOIR', 'lambda') * 1e-9  # m
diam = CONFIG_PASTIS.getfloat('LUVOIR', 'diameter')  # m
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9   # m

In [ ]:
im_lamD = CONFIG_PASTIS.getfloat('LUVOIR', 'im_size_lamD_hcipy')  # image size in lambda/D
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')

In [ ]:
print('LUVOIR apodizer design: {}'.format(coronagraph_design)) 
print()
print('Wavelength: {} m'.format(wvln))
print('Telescope diameter: {} m'.format(diam))
print('Number of segments: {}'.format(nb_seg))
print()
print('Image size: {} lambda/D'.format(im_lamD))
print('Sampling: {} px per lambda/D'.format(sampling))

In [ ]:
os.makedirs(resDir, exist_ok=True)
os.makedirs(os.path.join(resDir, 'OTE_images'), exist_ok=True)
os.makedirs(os.path.join(resDir, 'psfs'), exist_ok=True)

In [ ]:
util.copy_config(resDir)

In [ ]:
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, coronagraph_design, sampling) 

In [ ]:
luvoir.create_global_zernike_mirror(max_LO)
luvoir.create_segmented_mirror(max_MID)
luvoir.create_ripple_mirror(max_HI)
luvoir.create_continuous_deformable_mirror(num_DM_act)

In [ ]:
n_LO = luvoir.zernike_mirror.num_actuators 
n_MID = luvoir.sm.num_actuators 
n_HI = luvoir.ripple_mirror.num_actuators 
n_DM = luvoir.dm.num_actuators

In [ ]:
z_pup_downsample = CONFIG_PASTIS.getfloat('numerical', 'z_pup_downsample') 
N_pup_z = np.int(luvoir.pupil_grid.shape[0] / z_pup_downsample)
grid_zernike = hcipy.field.make_pupil_grid(N_pup_z, diameter=luvoir.diam)

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [ ]:
luvoir.zernike_mirror.actuators = LO_modes 
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes 
luvoir.dm.actuators = DM_modes

In [ ]:
unaberrated_coro_psf, ref = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate=None, norm_one_photon=False)

In [ ]:
norm = np.max(ref)

In [ ]:
dh_intensity = (unaberrated_coro_psf / norm) * luvoir.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(luvoir.dh_mask != 0)])
print('contrast floor: {}'.format(contrast_floor))

In [ ]:
nonaberrated_coro_psf, ref, efield = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
print('Generating the Efield for LO modes to science plane')
print('number of LO modes'.format(n_LO))

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [ ]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0,n_LO):
    print(pp)
    LO_modes = np.zeros(n_LO)
    LO_modes[pp] = nm_aber / 2
    luvoir.zernike_mirror.actuators  = LO_modes
    aberrated_coro_psf, ref, inter = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
mat_fast = np.zeros([n_LO, n_LO])
for i in range(0, n_LO):
    for j in range(0, n_LO):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [ ]:
matrix_pastis = np.copy(mat_fast) 

In [ ]:
matrix_pastis /= np.square(nm_aber * 1e9)

In [ ]:
filename_matrix = 'PASTISmatrix_num_LO_' + str(max_LO)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Re_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Im_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()

In [ ]:
print('Runtime for LO modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

In [ ]:
print('Generating the Efield for MID modes to science plane')
print('number of MID modes'.format(n_MID))

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [ ]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0,n_MID):
    print(pp)
    MID_modes = np.zeros(n_MID)
    MID_modes[pp] = nm_aber / 2
    luvoir.sm.actuators  = MID_modes
    aberrated_coro_psf, ref, inter = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
mat_fast = np.zeros([n_MID, n_MID])
for i in range(0, n_MID):
    for j in range(0, n_MID):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [ ]:
matrix_pastis = np.copy(mat_fast)
matrix_pastis /= np.square(nm_aber * 1e9)  

In [ ]:
filename_matrix = 'PASTISmatrix_num_MID_' + str(max_MID)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))


In [ ]:
filename_matrix = 'EFIELD_Re_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Im_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))


In [ ]:
end_time = time.time()
print('Runtime for MID modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

In [ ]:
print('Generating the Efield for HI modes to science plane')
print('number of HI modes'.format(n_HI))

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [ ]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0, n_HI):
    print(pp)
    HI_modes = np.zeros(n_HI)
    HI_modes[pp] = nm_aber / 2
    luvoir.ripple_mirror.actuators = HI_modes
    aberrated_coro_psf, ref, inter = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield')
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
mat_fast = np.zeros([n_HI, n_HI])
for i in range(0, n_HI):
    for j in range(0, n_HI):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_fast[i, j] = contrast

In [ ]:
matrix_pastis = np.copy(mat_fast) 
matrix_pastis /= np.square(nm_aber * 1e9)

In [ ]:
filename_matrix = 'PASTISmatrix_num_HI_' + str(max_HI)
hcipy.write_fits(matrix_pastis, os.path.join(resDir, filename_matrix + '.fits'))
print('Matrix saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Re_matrix_num_HI_' + str(max_HI)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_Im_matrix_num_HI_' + str(max_HI)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()
print('Runtime for HI modes:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

In [ ]:
print('Generating the Efield for LO modes to LOWFS')
print('number of LO modes'.format(n_LO))

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)

In [ ]:
luvoir.zernike_mirror.actuators = LO_modes 
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes 
luvoir.dm.actuators = DM_modes

In [ ]:
zernike_ref = luvoir.calc_low_order_wfs()

In [ ]:
zernike_ref_sub_real = hcipy.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hcipy.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref = zernike_ref_sub_real + 1j*zernike_ref_sub_imag

In [ ]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0, n_LO):
    print(pp)
    LO_modes = np.zeros(n_LO)
    LO_modes[pp] = nm_aber / 2
    luvoir.zernike_mirror.actuators = LO_modes
    zernike_meas = luvoir.calc_low_order_wfs()
    zernike_meas_sub_real = hcipy.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike,statistic='mean')
    zernike_meas_sub_imag = hcipy.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike,statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
filename_matrix = 'EFIELD_LOWFS_Re_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_LOWFS_Im_matrix_num_LO_' + str(max_LO)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()
print('Runtime for LO modes and LOWFS:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))

In [ ]:
print('Generating the Efield for MID modes to OBWFS')
print('number of MID modes'.format(n_MID))

In [ ]:
LO_modes = np.zeros(n_LO)
MID_modes = np.zeros(n_MID)
HI_modes = np.zeros(n_HI)
DM_modes = np.zeros(n_DM)
luvoir.zernike_mirror.actuators = LO_modes
luvoir.sm.actuators = MID_modes
luvoir.ripple_mirror.actuators = HI_modes
luvoir.dm.actuators = DM_modes

In [ ]:
zernike_ref = luvoir.calc_out_of_band_wfs()
zernike_ref_sub_real = hcipy.field.subsample_field(zernike_ref.real, z_pup_downsample, grid_zernike, statistic='mean')
zernike_ref_sub_imag = hcipy.field.subsample_field(zernike_ref.imag, z_pup_downsample, grid_zernike, statistic='mean')
Efield_ref = zernike_ref_sub_real + 1j*zernike_ref_sub_imag

In [ ]:
print('nm_aber: {} m'.format(nm_aber))
start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

In [ ]:
for pp in range(0, n_MID):
    print(pp)
    MID_modes = np.zeros(n_MID)
    MID_modes[pp] = nm_aber / 2
    luvoir.sm.actuators = MID_modes
    zernike_meas = luvoir.calc_out_of_band_wfs()
    zernike_meas_sub_real = hcipy.field.subsample_field(zernike_meas.real, z_pup_downsample, grid_zernike, statistic='mean')
    zernike_meas_sub_imag = hcipy.field.subsample_field(zernike_meas.imag, z_pup_downsample, grid_zernike, statistic='mean')
    focus_field1 = zernike_meas_sub_real + 1j * zernike_meas_sub_imag
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Re_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Re, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Real saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
filename_matrix = 'EFIELD_OBWFS_Im_matrix_num_MID_' + str(max_MID)
hcipy.write_fits(focus_fieldS_Im, os.path.join(resDir, filename_matrix + '.fits'))
print('Efield Imag saved to:', os.path.join(resDir, filename_matrix + '.fits'))

In [ ]:
end_time = time.time()
print('Runtime for MID modes and OBWFS:', end_time - start_time, 'sec =', (end_time - start_time) / 60, 'min')
print('Data saved to {}'.format(resDir))